In [5]:
import math
import numpy as np
import torch
import torchvision.transforms as T
from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoConfig, AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

def split_model(model_name):
    device_map = {}
    world_size = torch.cuda.device_count()
    config = AutoConfig.from_pretrained(path, trust_remote_code=True)
    num_layers = config.llm_config.num_hidden_layers
    # Since the first GPU will be used for ViT, treat it as half a GPU.
    num_layers_per_gpu = math.ceil(num_layers / (world_size - 0.5))
    num_layers_per_gpu = [num_layers_per_gpu] * world_size
    num_layers_per_gpu[0] = math.ceil(num_layers_per_gpu[0] * 0.5)
    layer_cnt = 0
    for i, num_layer in enumerate(num_layers_per_gpu):
        for j in range(num_layer):
            device_map[f'language_model.model.layers.{layer_cnt}'] = i
            layer_cnt += 1
    device_map['vision_model'] = 0
    device_map['mlp1'] = 0
    device_map['language_model.model.tok_embeddings'] = 0
    device_map['language_model.model.embed_tokens'] = 0
    device_map['language_model.output'] = 0
    device_map['language_model.model.norm'] = 0
    device_map['language_model.model.rotary_emb'] = 0
    device_map['language_model.lm_head'] = 0
    device_map[f'language_model.model.layers.{num_layers - 1}'] = 0

    return device_map

# video multi-round conversation (视频多轮对话) utils.
def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list

# If you set `load_in_8bit=True`, you will need two 80GB GPUs.
# If you set `load_in_8bit=False`, you will need at least three 80GB GPUs.
path = 'OpenGVLab/InternVL3-1B'
device_map = split_model('InternVL3-1B')
model = AutoModel.from_pretrained(
    path,
    torch_dtype=torch.bfloat16,
    load_in_8bit=False,
    low_cpu_mem_usage=True,
    use_flash_attn=True,
    trust_remote_code=True,
    device_map=device_map).eval()
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, use_fast=False)

generation_config = dict(max_new_tokens=1024, do_sample=True, temperature=0.7, top_p=0.95, top_k=10)

/home/paperspace/miniconda3/envs/25-07-11-internvl-2/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/paperspace/miniconda3/envs/25-07-11-internvl-2/lib/python3.9/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [18]:
# pure-text conversation (纯文本多轮对话)
question = '你好吗'
response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: 你好吗
Assistant: 你好！看起来我可能有点忙碌，所以没有实时的即时消息或回应。请随时告诉我你需要什么帮助或信息，我会尽力为你服务！


In [3]:
# set the max number of tiles in `max_num`
pixel_values = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()

# single-image single-round conversation (单图单轮对话)
question = '<image>\nPlease describe the image shortly.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: <image>
Please describe the image shortly.
Assistant: The image features a red panda with distinctive brown and white fur, resting on a piece of wood in a natural setting with green foliage in the background. The red panda appears relaxed and is looking directly at the camera.


In [6]:
subtitle_text = '那就不见不散了啊，哎，好，再见'

video_path = '/home/paperspace/Downloads/25-07-11-affectgpt-dataset-mini100/video/sample_00000007.mp4'
pixel_values, num_patches_list = load_video(video_path, num_segments=10, max_num=1)
pixel_values = pixel_values.to(torch.bfloat16).cuda()
video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
question = video_prefix + '<subtitle>\n' + subtitle_text + '\n</subtitle>\n' + 'Please identify the emotional states of the main character with specific reasons.'
# Frame1: <image>\nFrame2: <image>\n...\nFrame8: <image>\n{question}
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: Frame1: <image>
Frame2: <image>
Frame3: <image>
Frame4: <image>
Frame5: <image>
Frame6: <image>
Frame7: <image>
Frame8: <image>
Frame9: <image>
Frame10: <image>
<subtitle>
那就不见不散了啊，哎，好，再见
</subtitle>
Please identify the emotional states of the main character with specific reasons.
Assistant: The woman appears to be in a cheerful and happy mood. She smiles while holding the phone to her ear, indicating a sense of relief or satisfaction with the conversation. Her facial expressions suggest she is enjoying the conversation, further emphasizing the positive emotion she is conveying.


In [7]:
question = """Please extract emotion words into a list. Please separate different emotional categories with commas and output only the clearly identifiable emotional categories in a list format. If none are identified, please output an empty list. The output words have to be adjectives of feelings from those emotion words (internal state), not an action/expression (external manifestation).

EXAMPLES:
✓ Good: ["happy", "excited", "nervous"]
✗ Bad: ["smiling", "jumping", "talking loudly"]
"""
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


User: Please extract emotion words into a list. Please separate different emotional categories with commas and output only the clearly identifiable emotional categories in a list format. If none are identified, please output an empty list. The output words have to be adjectives of feelings from those emotion words (internal state), not an action/expression (external manifestation).

EXAMPLES:
✓ Good: ["happy", "excited", "nervous"]
✗ Bad: ["smiling", "jumping", "talking loudly"]

Assistant: , ["happy", "relieved", "smiling", "joyful"]


In [11]:
from datasets import load_dataset
from huggingface_hub import list_repo_files
import pandas as pd
import os

In [14]:
def create_enhanced_merged_dataset(local_path=None):
    """
    Create enhanced merged dataset with emotions, reasoning, and subtitle data.
    
    Args:
        local_path (str, optional): Path to local folder containing CSV files.
                                   If provided and all required files exist, 
                                   will read from local files instead of Hugging Face.
    
    Returns:
        dict: Dictionary containing:
            - enhanced_merged_by_name: Complete dataset with all merged data
            - merged_by_name: Basic merged dataset (track2 + track3)
            - subtitle_by_name: Subtitle data indexed by name
            - datasets: All loaded individual datasets
    """
    
    # Required CSV files
    required_files = [
        "subtitle_chieng.csv",
        "track2_train_mercaptionplus.csv", 
        "track2_train_ovmerd.csv",
        "track3_train_mercaptionplus.csv",
        "track3_train_ovmerd.csv"
    ]
    
    # Check if we should use local files
    use_local = False
    if local_path and os.path.exists(local_path):
        # Check if all required files exist in the local path
        local_files_exist = all(
            os.path.exists(os.path.join(local_path, file)) 
            for file in required_files
        )
        if local_files_exist:
            use_local = True
            csv_files = required_files
            print(f"✓ Using local files from: {local_path}")
        else:
            missing_files = [
                file for file in required_files 
                if not os.path.exists(os.path.join(local_path, file))
            ]
            print(f"❌ Missing files in {local_path}: {missing_files}")
            print("Falling back to Hugging Face...")
    
    # If not using local files, discover files using Hugging Face Hub API
    if not use_local:
        try:
            # List all files in the dataset repository
            files = list_repo_files("MERChallenge/MER2025", repo_type="dataset")
            
            # Filter for CSV files
            csv_files = [f for f in files if f.endswith('.csv')]
            
        except Exception as e:
            # Fallback to the required files list
            csv_files = required_files

    datasets = {}
    for csv_file in csv_files:
        try:
            dataset_name = csv_file.replace('.csv', '')
            
            if use_local:
                # Load from local file
                file_path = os.path.join(local_path, csv_file)
                df = pd.read_csv(file_path, encoding='utf-8')
                
                # Convert to datasets format
                from datasets import Dataset
                dataset_dict = {"train": Dataset.from_pandas(df)}
                datasets[dataset_name] = dataset_dict
                
            else:
                # Load from Hugging Face
                if csv_file == "subtitle_chieng.csv":
                    # Special handling for subtitle_chieng.csv which has encoding issues with datasets library
                    from huggingface_hub import hf_hub_download
                    
                    # Download the file first
                    file_path = hf_hub_download(
                        repo_id="MERChallenge/MER2025",
                        filename=csv_file,
                        repo_type="dataset"
                    )
                    
                    # Load with pandas
                    df = pd.read_csv(file_path, encoding='utf-8')
                    
                    # Convert to datasets format
                    from datasets import Dataset
                    dataset_dict = {"train": Dataset.from_pandas(df)}
                    datasets[dataset_name] = dataset_dict
                    
                else:
                    # Load without specifying split to see all available splits
                    dataset_dict = load_dataset(
                        "MERChallenge/MER2025", 
                        data_files=csv_file
                    )
                    datasets[dataset_name] = dataset_dict
                
        except Exception as e:
            pass  # Silently skip failed loads

    merged_by_name = {}
    
    # Check if we have both datasets
    if ('track2_train_mercaptionplus' in datasets and 
        'track3_train_mercaptionplus' in datasets):
        
        # Get the train splits from both datasets
        track2_data = datasets['track2_train_mercaptionplus']['train']
        track3_data = datasets['track3_train_mercaptionplus']['train']
        
        # Convert to pandas DataFrames for easier merging
        track2_df = track2_data.to_pandas()
        track3_df = track3_data.to_pandas()
        
        # Check if both have 'name' column
        if 'name' in track2_df.columns and 'name' in track3_df.columns:
            # Merge on 'name' column
            merged_df = pd.merge(
                track2_df, 
                track3_df, 
                on='name', 
                how='inner',  # Only keep rows that exist in both datasets
                suffixes=('_track2', '_track3')
            )
            
            # Create a list of dictionaries with combined data
            merged_data = []
            for _, row in merged_df.iterrows():
                combined_row = {
                    'name': row['name'],
                    'openset': row.get('openset', None),  # from track2
                    'reason': row.get('reason', None)     # from track3
                }
                
                # Add all other columns from both datasets
                for col in merged_df.columns:
                    if col not in ['name', 'openset', 'reason']:
                        combined_row[col] = row[col]
                
                merged_data.append(combined_row)
            
            # Create a dictionary indexed by name for easy access
            merged_by_name = {}
            for entry in merged_data:
                name = entry['name']
                merged_by_name[name] = entry

    subtitle_by_name = {}
    enhanced_merged_by_name = {}
    
    if 'subtitle_chieng' in datasets:
        subtitle_data = datasets['subtitle_chieng']['train']
        
        # Convert to pandas DataFrame
        subtitle_df = subtitle_data.to_pandas()
        
        # Create subtitle dictionary indexed by name
        subtitle_by_name = {}
        for _, row in subtitle_df.iterrows():
            name = row['name']
            subtitle_by_name[name] = {
                'name': name,
                'chinese': row['chinese'] if pd.notna(row['chinese']) else None,
                'english': row['english'] if pd.notna(row['english']) else None
            }
        
        # Try to merge subtitle data with existing merged data if available
        if merged_by_name:
            # Add subtitle data to existing merged data
            enhanced_merged_by_name = {}
            for name, data in merged_by_name.items():
                enhanced_data = data.copy()
                
                # Add subtitle information if available
                if name in subtitle_by_name:
                    subtitle_info = subtitle_by_name[name]
                    enhanced_data.update({
                        'chinese_subtitle': subtitle_info['chinese'],
                        'english_subtitle': subtitle_info['english']
                    })
                else:
                    enhanced_data.update({
                        'chinese_subtitle': None,
                        'english_subtitle': None
                    })
                
                enhanced_merged_by_name[name] = enhanced_data
            
            # Show sample of enhanced data
            print(f"Sample of enhanced data with subtitles:")
            sample_names_with_subtitles = [name for name in list(enhanced_merged_by_name.keys())[:5] 
                                         if enhanced_merged_by_name[name]['chinese_subtitle'] or enhanced_merged_by_name[name]['english_subtitle']]
            
            for name in sample_names_with_subtitles[:2]:
                data = enhanced_merged_by_name[name]
                print(f"  Name: {name}")
                print(f"    openset: {data.get('openset', 'N/A')}")
                print(f"    reason: {data.get('reason', 'N/A')[:100]}..." if data.get('reason') and len(data.get('reason', '')) > 100 else f"    reason: {data.get('reason', 'N/A')}")
                print(f"    chinese_subtitle: {data.get('chinese_subtitle', 'N/A')}")
                print(f"    english_subtitle: {data.get('english_subtitle', 'N/A')}")
                print()
            
    else:
        # If no subtitle data, use merged data as enhanced data
        if merged_by_name:
            enhanced_merged_by_name = merged_by_name.copy()
    
    # Return all the created data structures
    return enhanced_merged_by_name


In [18]:
enhanced_merged_by_name = create_enhanced_merged_dataset(local_path="/home/paperspace/Downloads/25-07-11-affectgpt-dataset-mini100")

print(enhanced_merged_by_name[list(enhanced_merged_by_name.keys())[0]])

✓ Using local files from: /home/paperspace/Downloads/25-07-11-affectgpt-dataset-mini100
Sample of enhanced data with subtitles:
  Name: samplenew3_00000058
    openset: [concern, pessimism]
    reason: In the text, the subtitle reads: "Given his precarious situation." Based on the description of the i...
    chinese_subtitle: 凭他的处境不妙
    english_subtitle: Given his situation is not good.

  Name: samplenew3_00000084
    openset: [concern, advice, care, dissatisfaction, worry]
    reason: In the text, the subtitle content says, "Please stop drinking." This sentence is likely spoken by th...
    chinese_subtitle: 别喝了好吗
    english_subtitle: Please don't drink anymore.

{'name': 'samplenew3_00000058', 'openset': '[concern, pessimism]', 'reason': 'In the text, the subtitle reads: "Given his precarious situation." Based on the description of the individual\'s trembling voice and tone indicating feeling overwhelmed or under a lot of pressure in the audio clues, as well as the calm and compos

In [19]:
import json
from datetime import datetime

# Initialize results storage
results = {}
processed_count = 0
error_count = 0

# Process each sample
for enhanced_merged_by_name_key in list(enhanced_merged_by_name.keys()):
    try:
        sample_data = enhanced_merged_by_name[enhanced_merged_by_name_key]
        
        # Extract required fields
        subtitle_text = sample_data.get('chinese_subtitle', '')
        video_name = sample_data['name']
        video_path = f'/home/paperspace/Downloads/25-07-11-affectgpt-dataset-mini100/video/{video_name}.mp4'
        
        # Skip if no subtitle
        if not subtitle_text:
            print(f"⚠️  Skipping {video_name}: No Chinese subtitle")
            continue
            
        print(f"\n[{processed_count + 1}] Processing: {video_name}")
        
        # Load and process video
        pixel_values, num_patches_list = load_video(video_path, num_segments=10, max_num=1)
        pixel_values = pixel_values.to(torch.bfloat16).cuda()
        video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
        
        # Emotional state analysis
        question1 = video_prefix + '<subtitle>\n' + subtitle_text + '\n</subtitle>\n' + 'Please identify the emotional states of the main character with specific reasons.'
        response1, history = model.chat(tokenizer, pixel_values, question1, generation_config,
                                       num_patches_list=num_patches_list, history=None, return_history=True)
        
        # Emotion word extraction
        question2 = """Please extract emotion words into a list. Please separate different emotional categories with commas and output only the clearly identifiable emotional categories in a list format. If none are identified, please output an empty list. The output words have to be adjectives of feelings from those emotion words (internal state), not an action/expression (external manifestation).

EXAMPLES:
✓ Good: ["happy", "excited", "nervous"]
✗ Bad: ["smiling", "jumping", "talking loudly"]
"""
        response2, history = model.chat(tokenizer, pixel_values, question2, generation_config, 
                                       history=history, return_history=True)
        
        # Store results
        results[video_name] = {
            'subtitle': subtitle_text,
            'emotional_analysis': response1,
            'emotion_words': response2,
            'openset': sample_data.get('openset', ''),
            'reason': sample_data.get('reason', ''),
            'processed_at': datetime.now().isoformat()
        }
        
        processed_count += 1
        print(f"✅ Completed: {video_name}")
        
        # Print progress every 10 samples
        if processed_count % 10 == 0:
            print(f"\n📊 Progress: {processed_count} samples processed")
            
    except FileNotFoundError:
        print(f"❌ File not found: {video_path}")
        error_count += 1
        
    except Exception as e:
        print(f"❌ Error processing {video_name}: {str(e)}")
        error_count += 1

# Save results to file
with open('emotion_analysis_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n🎉 Processing complete!")
print(f"✅ Successfully processed: {processed_count} samples")
print(f"❌ Errors encountered: {error_count} samples")
print(f"💾 Results saved to: emotion_analysis_results.json")


[1] Processing: samplenew3_00000058


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Completed: samplenew3_00000058

[2] Processing: samplenew3_00000084


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Completed: samplenew3_00000084

[3] Processing: samplenew3_00000139


...

✅ Completed: samplenew3_00000829

[49] Processing: samplenew3_00000839


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Completed: samplenew3_00000839

[50] Processing: samplenew3_00000891


Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151645 for open-end generation.


✅ Completed: samplenew3_00000891

📊 Progress: 50 samples processed

🎉 Processing complete!
✅ Successfully processed: 50 samples
❌ Errors encountered: 0 samples
💾 Results saved to: emotion_analysis_results.json


In [ ]:
# set the max number of tiles in `max_num`
pixel_values = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()
generation_config = dict(max_new_tokens=1024, do_sample=True)

# pure-text conversation (纯文本对话)
question = 'Hello, who are you?'
response, history = model.chat(tokenizer, None, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'Can you tell me a story?'
response, history = model.chat(tokenizer, None, question, generation_config, history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

# single-image single-round conversation (单图单轮对话)
question = '<image>\nPlease describe the image shortly.'
response = model.chat(tokenizer, pixel_values, question, generation_config)
print(f'User: {question}\nAssistant: {response}')

# single-image multi-round conversation (单图多轮对话)
question = '<image>\nPlease describe the image in detail.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'Please write a poem according to the image.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config, history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

# multi-image multi-round conversation, combined images (多图多轮对话，拼接图像)
pixel_values1 = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()
pixel_values2 = load_image('./examples/image2.jpg', max_num=12).to(torch.bfloat16).cuda()
pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)

question = '<image>\nDescribe the two images in detail.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'What are the similarities and differences between these two images.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

# multi-image multi-round conversation, separate images (多图多轮对话，独立图像)
pixel_values1 = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()
pixel_values2 = load_image('./examples/image2.jpg', max_num=12).to(torch.bfloat16).cuda()
pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)
num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]

question = 'Image-1: <image>\nImage-2: <image>\nDescribe the two images in detail.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list,
                               history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'What are the similarities and differences between these two images.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list,
                               history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')

# batch inference, single image per sample (单图批处理)
pixel_values1 = load_image('./examples/image1.jpg', max_num=12).to(torch.bfloat16).cuda()
pixel_values2 = load_image('./examples/image2.jpg', max_num=12).to(torch.bfloat16).cuda()
num_patches_list = [pixel_values1.size(0), pixel_values2.size(0)]
pixel_values = torch.cat((pixel_values1, pixel_values2), dim=0)

questions = ['<image>\nDescribe the image in detail.'] * len(num_patches_list)
responses = model.batch_chat(tokenizer, pixel_values,
                             num_patches_list=num_patches_list,
                             questions=questions,
                             generation_config=generation_config)
for question, response in zip(questions, responses):
    print(f'User: {question}\nAssistant: {response}')

# video multi-round conversation (视频多轮对话)
def get_index(bound, fps, max_frame, first_idx=0, num_segments=32):
    if bound:
        start, end = bound[0], bound[1]
    else:
        start, end = -100000, 100000
    start_idx = max(first_idx, round(start * fps))
    end_idx = min(round(end * fps), max_frame)
    seg_size = float(end_idx - start_idx) / num_segments
    frame_indices = np.array([
        int(start_idx + (seg_size / 2) + np.round(seg_size * idx))
        for idx in range(num_segments)
    ])
    return frame_indices

def load_video(video_path, bound=None, input_size=448, max_num=1, num_segments=32):
    vr = VideoReader(video_path, ctx=cpu(0), num_threads=1)
    max_frame = len(vr) - 1
    fps = float(vr.get_avg_fps())

    pixel_values_list, num_patches_list = [], []
    transform = build_transform(input_size=input_size)
    frame_indices = get_index(bound, fps, max_frame, first_idx=0, num_segments=num_segments)
    for frame_index in frame_indices:
        img = Image.fromarray(vr[frame_index].asnumpy()).convert('RGB')
        img = dynamic_preprocess(img, image_size=input_size, use_thumbnail=True, max_num=max_num)
        pixel_values = [transform(tile) for tile in img]
        pixel_values = torch.stack(pixel_values)
        num_patches_list.append(pixel_values.shape[0])
        pixel_values_list.append(pixel_values)
    pixel_values = torch.cat(pixel_values_list)
    return pixel_values, num_patches_list

video_path = './examples/red-panda.mp4'
pixel_values, num_patches_list = load_video(video_path, num_segments=8, max_num=1)
pixel_values = pixel_values.to(torch.bfloat16).cuda()
video_prefix = ''.join([f'Frame{i+1}: <image>\n' for i in range(len(num_patches_list))])
question = video_prefix + 'What is the red panda doing?'
# Frame1: <image>\nFrame2: <image>\n...\nFrame8: <image>\n{question}
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=None, return_history=True)
print(f'User: {question}\nAssistant: {response}')

question = 'Describe this video in detail.'
response, history = model.chat(tokenizer, pixel_values, question, generation_config,
                               num_patches_list=num_patches_list, history=history, return_history=True)
print(f'User: {question}\nAssistant: {response}')